In [1]:
import numpy as np 
import pandas as pd
import copy as copy
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import mixture
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm

In [2]:
def lasso_selection(X_ls,y_ls,alpha,n):
    preprocess = make_pipeline(SimpleImputer(missing_values=np.nan, strategy='median'), 
                               #PCA(n_components=X_ls.shape[1], whiten=True), 
                               StandardScaler())
    X_pre = preprocess.fit_transform(X_ls)
    Y_pre = StandardScaler().fit_transform(y_ls)
    lasso = Lasso(alpha=alpha, max_iter=100000)
    lasso.fit(X_pre, Y_pre) 
    lasso_coefs = lasso.coef_
    coef = pd.DataFrame(lasso_coefs, index=X_ls.columns)
    best_col = abs(coef).nlargest(n=n,columns=0).index
    return best_col

In [3]:
def outlier_detection(X_train_od, y_train_od, n_PCA, quantile):
    
    # project using PCA
    clf = make_pipeline(SimpleImputer(missing_values=np.nan, strategy='median'), 
                        StandardScaler(), 
                        PCA(n_components=n_PCA, whiten=False))
    # density estimation with GMM
    gmm = mixture.GaussianMixture(n_components=1, covariance_type='full')

    anomaly_detected = True
    iter = 0
    cutoff=np.nan
    anom_id=set()
    df_total_od=pd.concat([X_train_od,y_train_od], axis=1)
    
    # method : estimate density (but contaminated with outliers), remove outliers, 
    #          re-estimate density without outliers, remove (newly detected) outliers,
    #          ...
    # loop until no more outliers are detected
    while anomaly_detected:
        iter+=1
        
        # extract features using PCA
        feat = pd.DataFrame(clf.fit_transform(df_total_od), index=df_total_od.index)
        
        # compute log-likelihood of observed data
        gmm.fit(feat)
        loglike = gmm.score_samples(feat)
        
        # set the cutoff at the first iteration
        if iter==1:
            cutoff = np.quantile(loglike, quantile)
        
        # add newly detected anomalies to the set of all anomalies
        anom_id=anom_id.union(set(df_total_od.index[(loglike <= cutoff)]))
        
        # number of newly detected outliers
        n_anom = df_total_od.shape[0] - (loglike > cutoff).sum()
        if n_anom==0:
            anomaly_detected = False
        #print("Number of removed outliers: ", n_anom)
        
        # remove newly detected outliers
        df_total_od=df_total_od[loglike > cutoff]
        
        
    return anom_id

In [4]:
def process(X_train, X_test, Y_train, alpha, lasso_nlargest, n_PCA_anom, quantile_anom):
    max_iter=5
    X=copy.copy(X_train)
    Y=copy.copy(Y_train)
    
    # idea : select best features, remove outliers (or vice-versa)
    # problem : outliers could influence the selection of best features and then make the anomaly detection more difficult
    # solution : loop on feature selection and anomaly detection to remove influence of outliers on feature selection
    for i in range(max_iter):
        # select best features (based on data contaminated by outliers during first round, then without outliers in following runs)
        col = lasso_selection(X,Y,alpha,lasso_nlargest)
        X_train_red = X_train[X_train.columns.intersection(col)]
        
        # detect outliers based on features
        anom_id = outlier_detection(X_train_red, Y_train, n_PCA=n_PCA_anom, quantile=quantile_anom)
        
        # remove outliers from original data
        X=copy.copy(X_train)
        Y=copy.copy(Y_train)
        for id in anom_id:
            X = X.drop(id)
            Y = Y.drop(id)
        
        # stop if no change 
        if (i>0 and set(col_prev)==set(col) and anom_id_prev==anom_id):
            break
        col_prev = col
        anom_id_prev = anom_id
    # print(f'Number of anomalies detected: {len(anom_id)}')
    
    # once outliers are removed, we select best features
    col = lasso_selection(X,Y,alpha,lasso_nlargest)
    X_train_proc = X[X.columns.intersection(col)]
    Y_train_proc = Y
    X_test_proc = X_test[X_test.columns.intersection(col)]
    
    # we have removed outliers and selected best features but haven't done the final imputation yet
    return X_train_proc, X_test_proc, Y_train_proc



In [5]:
X_train = pd.read_csv('X_train.csv').drop(columns="id")
X_test = pd.read_csv('X_test.csv').drop(columns="id")

Y_train = pd.read_csv('Y_train.csv').drop(columns="id")


In [6]:
X_train = X_train.loc[:, X_train.var() != 0.0]
X_test = X_test.loc[:, X_test.var() != 0.0]

In [7]:
X_train_t = copy.copy(X_train)
Y_train_t = copy.copy(Y_train)
mask = np.arange(X_train_t.shape[0])
np.random.shuffle(mask)
X_train_t = X_train_t.loc[mask]
Y_train_t = Y_train_t.loc[mask]

In [8]:
alpha = 1.0/50.0 #50 #22
lasso_nlargest = 20 # 30 #50
n_PCA_anom = 20 #23
quantile_anom = 0.01
n_KNN = 6

print(f'param\t\tGBR\t\tLGB')

for param in np.linspace(0.01, 0.05, 5):
    quantile_anom = param

    step = X_train_t.shape[0] // 5
    r2_val_lgb, r2_train_lgb = 0, 0
    r2_val_gbr, r2_train_gbr = 0, 0
    
    for i in range(5):
        
        X_val, y_val = X_train_t.iloc[i*step:(i+1)*step], Y_train_t.iloc[i*step:(i+1)*step]
        X_train_CV = pd.concat((X_train_t.iloc[(i+1)*step:], X_train_t.iloc[:i*step]), axis=0)
        Y_train_CV = pd.concat((Y_train_t.iloc[(i+1)*step:], Y_train_t.iloc[:i*step]), axis=0)
        
        X_train_CV, X_val_proc, y_train_CV = process(X_train_CV, X_val, Y_train_CV, alpha, lasso_nlargest, int(n_PCA_anom), quantile_anom)
        
        # imputation 
        clf = make_pipeline(StandardScaler(), 
                            KNNImputer(missing_values=np.nan, n_neighbors=n_KNN), 
                            StandardScaler(), 
                            #PCA(n_components=int(n_PCA_feat), whiten=False),
                            )
        clf.fit(pd.concat([X_train_CV, X_val_proc], axis=0))
        X_train_CV = clf.transform(X_train_CV)
        X_val = clf.transform(X_val_proc)
        y_scaler = StandardScaler()
        y_train_CV = y_scaler.fit_transform(y_train_CV).ravel()
        
        lgb = lightgbm.LGBMRegressor(max_depth=2, learning_rate=0.015, n_estimators=500)
        gbr = GradientBoostingRegressor(max_depth=2, learning_rate=0.015, n_estimators=500)
        
        gbr.fit(X_train_CV, y_train_CV)
        lgb.fit(X_train_CV, y_train_CV)
        
        r2_val_lgb += r2_score(y_val, y_scaler.inverse_transform(lgb.predict(X_val).reshape(-1, 1)))
        r2_val_gbr += r2_score(y_val, y_scaler.inverse_transform(gbr.predict(X_val).reshape(-1, 1)))
        
        r2_train_lgb += r2_score(y_train_CV, lgb.predict(X_train_CV))
        r2_train_gbr += r2_score(y_train_CV, gbr.predict(X_train_CV))
        
    print(f'{param:5.3f}\t\t{r2_val_gbr/5:6.4f}\t\t{r2_val_lgb/5:6.4f}')
                        

param		GBR		LGB
0.010		0.4674		0.4753
0.020		0.4317		0.4431
0.030		0.4544		0.4470
0.040		0.4462		0.4539
0.050		0.4187		0.4279
